## 1. 遍历循环计算

In [1]:
import numpy as np
def getIoU(anchors,gts):
    N,_=anchors.shape
    M,_=gts.shape
    
    iou=np.zeros((N,M))
    
    anchors_area=(anchors[:,2]-anchors[:,0]+1)*(anchors[:,3]-anchors[:,1]+1)
    gts_area=(gts[:,2]-gts[:,0]+1)*(gts[:,3]-gts[:,1]+1)
    
    for i in range(N):
        for j in range(M):
            w = min(anchors[i,2],gts[j,2])-max(anchors[i,0],gts[j,0])
            if(w>0):
                h = min(anchors[i,3],gts[j,3])-max(anchors[i,1],gts[j,1])
                if(h>0):
                    intersection=(w+1)*(h+1)
                    union=anchors_area[i]+gts_area[j]-intersection
                    iou[i,j]=intersection/union
    return iou

##  2. 纯numpy计算

In [2]:
def np_iou(anchors,gts):
    N,_=anchors.shape
    M,_=gts.shape
    anchors_area=(anchors[:,2]-anchors[:,0]+1)*(anchors[:,3]-anchors[:,1]+1)
    gts_area=(gts[:,2]-gts[:,0]+1)*(gts[:,3]-gts[:,1]+1)
    
    # 扩维
    anchors=np.expand_dims(anchors,axis=1)
    gts=np.expand_dims(gts,axis=0)
    
    anchors_area=np.expand_dims(anchors_area,axis=1)
    gts_area=np.expand_dims(gts_area,axis=0)

    
    # 利用numpy广播
    iou=np.zeros((N,M,3))
    iou[:,:,0]= np.max(np.where(np.less(anchors[:,:,2],gts[:,:,2]),anchors[:,:,2],gts[:,:,2])-
                       np.where(np.greater(anchors[:,:,0],gts[:,:,0]),anchors[:,:,0],gts[:,:,0])+1,0)
    iou[:,:,1]= np.max(np.where(np.less(anchors[:,:,3],gts[:,:,3]),anchors[:,:,3],gts[:,:,3])-
                       np.where(np.greater(anchors[:,:,1],gts[:,:,1]),anchors[:,:,1],gts[:,:,1])+1,0)
    iou[:,:,2]=iou[:,:,0]*iou[:,:,1]/(anchors_area+gts_area-iou[:,:,0]*iou[:,:,1])
    return iou[:,:,2]

## 3.比较结果和耗时

In [3]:
gt=np.array([0,0,100,100])
gts=np.tile(gt,reps=(100,1))

anchor=np.array([50,50,150,150])
anchors=np.tile(anchor,reps=(10000,1))

ious=getIoU(anchors,gts)
np_ious=np_iou(anchors,gts)

In [4]:
np.allclose(ious,np_ious)

True

In [5]:
%timeit getIoU(anchors,gts)

1 loop, best of 3: 2.74 s per loop


In [6]:
%timeit np_iou(anchors,gts)

10 loops, best of 3: 57.3 ms per loop


#   二. 获取正负样本

In [23]:
def get_idxes(overlaps,pos_iou=0.7,neg_iou=0.3):
    
    
    
    n,m=overlaps.shape
    max_overlap_indices=np.argmax(overlaps,axis=1) # (n,)
    max_overlaps=overlaps[np.arange(n),max_overlap_indices] #  切记，这仍然是一维数组(n,)
    
    # anchor最大的iou>0.7的是正样本
    pos_indices=max_overlaps>pos_iou
    
    # gt 最大的
    gt_set=set(np.unique(max_overlap_indices[pos_indices]))
    
    no_match_gt=set(range(m))-gt_set  #  集合可以做差集
    
    gmax_overlap_indices=np.argmax(overlaps,axis=0)
    print(no_match_gt)
    for g in no_match_gt:
        idx=gmax_overlap_indices[g]
        print("g:{},idx:{}".format(g,idx))
        pos_indices[idx]=True
        max_overlap_indices[idx]=g
    
    # 负样本
    neg_indices=(max_overlaps<neg_iou) & ~pos_indices
    
    return pos_indices,neg_indices,max_overlap_indices
    

In [27]:
boxs=np.array([[0.1,0.2,0.2,0.1],
              [0.1,0.3,0.4,0.66],
               [0.1,0.8,0.4,0.6],
               [0.1,0.2,0.4,0.6],
               [0.2,0.2,0.9,0.6],
               [0.3,0.2,0.75,0.6],
               [0.4,0.2,0.4,0.5]
              ])
pos_indices,neg_indices,max_overlap_indices=get_idxes(boxs)

{0, 3}
g:0,idx:6
g:3,idx:1


In [28]:
pos_indices

array([False,  True,  True, False,  True,  True,  True])

# 三. 计算回归目标 

In [31]:
def tochw(boxes):
    nboxes=np.zeros_like(boxes)
    nboxes[:,0]=(boxes[:,0]+boxes[:,2])/2
    nboxes[:,1]=(boxes[:,1]+boxes[:,3])/2
    nboxes[:,2]=boxes[:,2]-boxes[:,0]
    nboxes[:,3]=boxes[:,3]-boxes[:,1]
    return nboxes

In [33]:
def rgs_target(anchors,gts):
    c_anchors=tochw(anchors)
    c_gts=tochw(gts)
    
    tx=(c_gts[:,0]-c_anchors[:,0])/c_anchors[:,2]
    ty=(c_gts[:,1]-c_anchors[:,1])/c_anchors[:,3]
    tw=np.log(c_gts[:,2]/c_anchors[:,2])
    th=np.log(c_gts[:,3]/c_anchors[:,3])
    
    return np.stack((tx,ty,tw,th)).T

In [34]:
np.stack((max_overlap_indices,max_overlap_indices))

array([[1, 3, 1, 3, 2, 2, 0],
       [1, 3, 1, 3, 2, 2, 0]], dtype=int64)

In [15]:
pos_indices[0]=True

In [35]:
x=np.random.randn(3,3)
indices=np.argmax(x,axis=1)
x

array([[-1.73202346, -0.43052661,  0.54634604],
       [-0.16545724,  1.58035509,  0.85180594],
       [ 0.31609746, -0.27810496, -0.60235744]])

In [57]:
a=np.where((x[:,0]>0) | (x[:,2]>0 & (x[:,1]>-1)))

In [60]:
np.delete(x,[1],axis=0)
x[[True,False,False]]

array([[-1.73202346, -0.43052661,  0.54634604]])

In [51]:
np.where(x>0)

(array([0, 1, 1, 2], dtype=int64), array([2, 1, 2, 0], dtype=int64))

In [13]:
x[range(3),indices]>
x

SyntaxError: invalid syntax (<ipython-input-13-0cc249c18505>, line 1)

In [ ]:
a=x[range(3),indices]>1

In [ ]:
x[range(3),indices]

In [ ]:
x

In [ ]:
x=list(['b', 'h', 'o', 'n', 'p', 't', 'y'])
if 'b' in x:
    print ('yes')

In [ ]:
set(np.arange(6))-set(range(3))